In [45]:
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import sentence_bleu
from bert_score import score as bscore
import rouge
from argparse import ArgumentParser
import yaml
import re
import torch
from sklearn.metrics.pairwise import cosine_similarity as cos_sim
from sentence_transformers import SentenceTransformer

# to do : evaluate with ibleu

In [ ]:
parser = ArgumentParser()
parser.add_argument("--config_path", dest="config_path",
                    default='../configs/dpng_transformer_bert_tokenizer_bow_indivtopk.yaml')

args = parser.parse_args()
config_path = args.config_path
print("config_path:", config_path)


In [22]:
# config_path = '../configs/base_transformer.yaml'
# config_path = '../configs/dpng_transformer.yaml'
# config_path = '../configs/dpng_transformer_bert_tokenizer_bow_indivtopk.yaml'
# config_path =  '../configs/dpng_transformer_bert_tokenizer_bow_indivtopk_onlybow.yaml'
config_path = '../configs/dpng_transformer_bert_attention.yaml'



In [23]:
with open(config_path) as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    print(config)    
    
prediction_path = config['test_output_file']
eval_log_path = prediction_path.replace('outputs', 'scores')

train_size = config['train_size']
val_size = config['val_size']
test_size = config['test_size']



{'save_model_path': '../models/transformer_bert_enc_attention.pth', 'log_file': '../logs/transformer_bert_enc_attention.txt', 'test_output_file': '../outputs/test_transformer_bert_enc_attention.txt', 'val_output_file': '../outputs/val_transformer_bert_enc_attention.txt', 'preprocessed_folder': '../data/preprocess_quora_bert_mask_predict/', 'dataset': 'quora_bert_dataset', 'num_epochs': 50, 'batch_size': 100, 'd_model': 450, 'd_inner_hid': 512, 'd_k': 50, 'd_v': 50, 'n_head': 9, 'n_layers': 3, 'n_warmup_steps': 12000, 'dropout': 0.1, 'embs_share_weight': True, 'proj_share_weight': True, 'label_smoothing': False, 'train_size': 100000, 'val_size': 4000, 'test_size': 20000, 'is_bow': False, 'lr': '1e-3'}


In [24]:
# for bert attention experiments
# prediction_path = '../outputs/tune/transformer_key_enc_bert_val_attention_alpha0.5.txt'
eval_log_path = prediction_path.replace('outputs', 'scores')

In [25]:
reference_path = '../data/quora_train.txt'

In [6]:
##### Arguments #####
# Arguments of DNPG

# train_size = 100000
# val_size = 4000
# test_size = 20000

# prediction_path = '../outputs/test_DNPG_transformer_out.txt'
# eval_log_path = '../scores/test_DNPG_transformer_out.txt'

# prediction_path = '../outputs/test_DNPG_transformer_bert_tokenizer_out.txt'
# eval_log_path = '../scores/test_DNPG_transformer_bert_tokenizer_out.txt'

# prediction_path = '../outputs/test_DNPG_transformer_bert_tokenizer_bow_out.txt'
# eval_log_path = '../scores/test_DNPG_transformer_bert_tokenizer_bow_out.txt'

# prediction_path = '../outputs/test_DNPG_transformer_bert_tokenizer_bow_indivtopk_out.txt'
# eval_log_path = '../scores/test_DNPG_transformer_bert_tokenizer_bow_indivtop_out.txt'

###################

In [26]:
# read reference sentence and prediction sentence
reference_text = open(reference_path, 'r').readlines()
reference_text = reference_text[train_size+val_size:train_size+val_size+test_size]
reference_text = [text.strip().split('\t')[1] for text in reference_text]

# normalize reference corpus , eg: seperate question mark , remove '(' ,')' etc
reference_text = [re.sub(r"([.!?])", r" \1", seq) for seq in reference_text]
reference_text = [text.lower() for text in reference_text]
# reference_text = [re.sub(r"[^a-zA-Z.!?]+", r" ", seq) for seq in reference_text]
reference_corpus = [[text.split()] for text in reference_text]

In [27]:
prediction_text = open(prediction_path, 'r').readlines()
prediction_text = [text.replace('Predict: ', '').strip().lower() for text in prediction_text if 'Predict: ' in text]
prediction_corpus = [text.split() for text in prediction_text]

In [28]:
print(reference_text[0], prediction_text[0], prediction_corpus[0])

is it healthy to eat fish every day ? what are some good side dishes for a healthy time ? ['what', 'are', 'some', 'good', 'side', 'dishes', 'for', 'a', 'healthy', 'time', '?']


In [29]:
print(len(reference_text))
print(len(prediction_text))

20000
20000


In [44]:
# i = 15
# print(reference_corpus[i])
# print(prediction_corpus[i])
# # sentence_bleu(reference_corpus[i], prediction_corpus[i], weights=(0.33, 0.33, 0.34, 0))
# sentence_bleu(reference_corpus[i], prediction_corpus[i], weights=(0.25, 0.25, 0.25, 0.25))

# # r = [['Why', 'is', 'chemistry', 'so', 'boring', '?']]
# # p = ['Why', 'is', 'chemistry', 'so', 'boring', '?']
# # sentence_bleu(r, p, weights=(0.25, 0.25, 0.25, 0.25))

In [77]:
# print(reference_corpus[:6])
# print(prediction_corpus[:6])

In [57]:
print("[Info] Calculating BLEU 1...")
bleu1 = corpus_bleu(reference_corpus, prediction_corpus, weights=(1, 0, 0, 0))
print("[Info] Calculating BLEU 2...")
bleu2 = corpus_bleu(reference_corpus, prediction_corpus, weights=(0.5, 0.5, 0, 0))
print("[Info] Calculating BLEU 3...")
bleu3 = corpus_bleu(reference_corpus, prediction_corpus, weights=(0.33, 0.33, 0.34, 0))
print("[Info] Calculating BLEU 4...")
bleu4 = corpus_bleu(reference_corpus, prediction_corpus, weights=(0.25, 0.25, 0.25, 0.25))
print("[Info] Done")

[Info] Calculating BLEU 1...
[Info] Calculating BLEU 2...
[Info] Calculating BLEU 3...
[Info] Calculating BLEU 4...
[Info] Done


In [58]:
print("[Info] BLEU1 Score: {}".format(bleu1))
print("[Info] BLEU2 Score: {}".format(bleu2))
print("[Info] BLEU3 Score: {}".format(bleu3))
print("[Info] BLEU4 Score: {}".format(bleu4))

[Info] BLEU1 Score: 0.4515410119541843
[Info] BLEU2 Score: 0.3122080569065922
[Info] BLEU3 Score: 0.2315023877412737
[Info] BLEU4 Score: 0.17811518172449067


In [68]:
rouge_evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l'], max_n=2)
rouge_scores = rouge_evaluator.get_scores(prediction_text, reference_text)

In [69]:
print("[Info] Rouge 1 score: {}".format(rouge_scores["rouge-1"]["f"]))
print("[Info] Rouge 2 score: {}".format(rouge_scores["rouge-2"]["f"]))
print("[Info] Rouge l score: {}".format(rouge_scores["rouge-l"]["f"]))


[Info] Rouge 1 score: 0.451004937367961
[Info] Rouge 2 score: 0.2397663124055765
[Info] Rouge l score: 0.4357079379192744


In [ ]:
# rouge
# lower
# [Info] Rouge 1 score: 0.451004937367961
# [Info] Rouge 2 score: 0.2397663124055765
# [Info] Rouge l score: 0.4357079379192744

# not lower

In [16]:
# save to file
f = open(eval_log_path, 'w')
f.write("[Info] BLEU1 Score: {}\n".format(bleu1))
f.write("[Info] BLEU2 Score: {}\n".format(bleu2))
f.write("[Info] BLEU3 Score: {}\n".format(bleu3))
f.write("[Info] BLEU4 Score: {}\n".format(bleu4))

f.write("\n\n[Info] Rouge 1 score: {}\n".format(rouge_scores["rouge-1"]["f"]))
f.write("[Info] Rouge 2 score: {}\n".format(rouge_scores["rouge-2"]["f"]))
f.write("[Info] Rouge l score: {}\n".format(rouge_scores["rouge-l"]["f"]))

f.close()

In [9]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [10]:
print(reference_text[:2], prediction_text[:2])

['is it healthy to eat fish every day ?', 'how do we write the exam ?'] ['what are some good side dishes for a healthy time ?', 'what are some good ways to improve my concentration ?']


In [15]:
P, R, F1 = bscore(prediction_text, reference_text, lang="en", model_type="bert-base-uncased", device=device, verbose=True)

calculating scores...
computing bert embedding.



computing greedy matching.



done in 42.46 seconds, 470.98 sentences/sec


In [20]:
print(F1.mean().item())

0.6827602386474609


In [30]:
model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')

In [46]:
pred_embeddings = model.encode(prediction_text)
ref_embeddings = model.encode(reference_text)

In [47]:
bertcs = []

In [48]:
for pred_emb, ref_emb in zip(pred_embeddings, ref_embeddings):
    sim = cos_sim(pred_emb.reshape(1, -1), ref_emb.reshape(1, -1))
    bertcs.append(sim[0][0])

In [51]:
import numpy as np
np.mean(bertcs)

0.7037093